<a href="https://colab.research.google.com/github/KevinTheRainmaker/Speech_Emotion_Classifier/blob/main/Model_loaded_as_HD5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
# from sklearn.preprocessing import OneHotEncoder

import librosa

In [24]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

drive_path = '/content/drive/MyDrive/data/emotional_speech'

Mounted at /content/drive


In [27]:
sample_path = (drive_path + '/RAVDESS/Actor_01/03-01-05-02-01-02-01.wav')

In [28]:
sample_path

'/content/drive/MyDrive/data/emotional_speech/RAVDESS/Actor_01/03-01-05-02-01-02-01.wav'

In [29]:
def extract_features(data):
  # ZCR
  result = np.array([])
  zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
  result=np.hstack((result, zcr)) # stacking horizontally

  # Chroma_stft
  stft = np.abs(librosa.stft(data))
  chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=22050).T, axis=0)
  result = np.hstack((result, chroma_stft))

  # MFCC
  mfcc = np.mean(librosa.feature.mfcc(y=data, sr=22050).T, axis=0)
  result = np.hstack((result, mfcc))

  # Root Mean Square Value
  rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
  result = np.hstack((result, rms))

  # MelSpectogram
  mel = np.mean(librosa.feature.melspectrogram(y=data, sr=22050).T, axis=0)
  result = np.hstack((result, mel))
  
  return result

In [30]:
def get_features(path):
  data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
  
  res1 = extract_features(data)
  result = np.array(res1)
  
  return result

In [31]:
model = tf.keras.models.load_model(os.path.join(drive_path, 'best-model-100.h5'))

In [32]:
# label = np.array(['angry','calm','disgust','fear','happy','neutral','sad','surprise'])
# encoder = OneHotEncoder()
# label = encoder.fit_transform(np.array(label).reshape(-1,1)).toarray()

In [33]:
feature = get_features(sample_path)
temp = np.expand_dims(feature, axis=1)
data = temp.reshape((1, temp.shape[0], temp.shape[1]))

In [34]:
pred = model.predict(data)
print(pred)
# result = encoder.inverse_transform(pred)
# result

[[0. 0. 0. 1. 0. 0. 0. 0.]]


# '😡', '🙂', '😩', '😱', '😊', '😐', '😭', '😲'